In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def velocity_Newtonian():
    pass

def velocity_Bingham():
    pass

def levee_wdith():
    pass

def channel_width():
    pass

def cooling_timescale():
    pass

def advancement_timescale():
    pass

def flow_length():
    pass

In [ ]:
def radiative_flux():
    pass

def conductive_flux():
    pass